In [3]:
%pip install -q timm


Note: you may need to restart the kernel to use updated packages.


In [2]:
# main.py
import argparse
# === One-cell runner (Known-T) ===
import sys, numpy as np
sys.path.append(".")  # ensure current folder is importable

In [4]:
print(mean, std)

0.9652666666666667 0.006336490441175714


In [2]:
from core import run_known_T, parse_T_str
NPZ = r"datasets/FashionMNIST0.3.npz"
T_str = "0.7,0.3,0, 0,0.7,0.3, 0.3,0,0.7"  # replace with your exact matrix if different

mean, std = run_known_T(
    npz_path=NPZ,
    T=parse_T_str(T_str),
    trials=5,          # use 10 for final
    grayscale=True,    # FashionMNIST
    model_name="vit_tiny_patch16_224",
    epochs=15,
    batch_size=128,
    loss_name="forward"  # or "gce"/"bootstrap" for your 2nd classifier
)
print(f"FashionMNIST0.3 ViT (forward-T): mean={mean:.4f} std={std:.4f}")


FashionMNIST0.3 ViT (forward-T): mean=0.9653 std=0.0063


In [1]:
from core import run_known_T, parse_T_str
NPZ = r"datasets/FashionMNIST0.6.npz"
T_str = "0.4,0.3,0.3, 0.3,0.4,0.3, 0.3,0.3,0.4"

mean, std = run_known_T(
    npz_path=NPZ, T=parse_T_str(T_str),
    trials=5, grayscale=True, epochs=15, batch_size=128,
    loss_name="forward"
)
print(f"FashionMNIST0.6 ViT (forward-T): mean={mean:.4f} std={std:.4f}")


C:\Users\William\desktop\comp5328-Assign2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FashionMNIST0.6 ViT (forward-T): mean=0.4227 std=0.1001


In [6]:
from core import run_unknown_T
NPZ = r"datasets/CIFAR.npz"

mean, std = run_unknown_T(
    npz_path=NPZ,
    trials=5,            # use 10 for final
    grayscale=False,     # CIFAR is RGB
    warmup_epochs=5,     # warm-up before estimating T̂
    epochs=15,
    batch_size=128
)
print(f"CIFAR ViT (estimate T̂): mean={mean:.4f} std={std:.4f}")


CIFAR ViT (estimate T̂): mean=0.4073 std=0.0252


In [6]:
NPZ_06 = r"datasets/FashionMNIST0.6.npz"
m1,s1 = run_known_T(NPZ_06, T,   trials=3, grayscale=True, epochs=6)
m2,s2 = run_known_T(NPZ_06, T.T, trials=3, grayscale=True, epochs=6)
print("T   ->", m1, s1, " |  T.T ->", m2, s2)


NameError: name 'T' is not defined

In [9]:
import importlib, core
importlib.reload(core)  # pick up any edits
NPZ_06 = r"datasets/FashionMNIST0.6.npz"

NPZ03 = r"datasets/FashionMNIST0.3.npz"
NPZ06 = r"datasets/FashionMNIST0.6.npz"


In [7]:
import numpy as np
from core import parse_T_str

T03_raw = parse_T_str("0.7,0.3,0, 0,0.7,0.3, 0.3,0,0.7")
NPZ06 = parse_T_str("0.4,0.3,0.3, 0.3,0.4,0.3, 0.3,0.3,0.4")

print("Row sums 0.3:", T03_raw.sum(axis=1))
print("Row sums 0.6:", T06_raw.sum(axis=1))


Row sums 0.3: [1. 1. 1.]
Row sums 0.6: [1. 1. 1.]


In [9]:
from core import run_known_T

m_T,  s_T  = run_known_T(NPZ06, T06_raw,   trials=3, grayscale=True, epochs=6, batch_size=128)
m_Tt, s_Tt = run_known_T(NPZ06, T06_raw.T, trials=3, grayscale=True, epochs=6, batch_size=128)
print("0.6 with T   ->", m_T,  s_T)
print("0.6 with T.T ->", m_Tt, s_Tt)


0.6 with T   -> 0.5448888888888889 0.07639096966863021
0.6 with T.T -> 0.5316666666666666 0.05099963689050115


In [11]:
m_gce, s_gce = run_known_T(
    npz_path=NPZ06,
    T=np.eye(3),          # no forward correction
    trials=10,
    grayscale=True,
    epochs=20,
    batch_size=128,
    loss_name="gce"       # or "bootstrap"
)
print("0.6 GCE baseline:", m_gce, s_gce)


0.6 GCE baseline: 0.611 0.05373349875905056


In [11]:
mean06, std06 = run_known_T(NPZ06, T=T06_raw, trials=10, grayscale=True, epochs=30, batch_size=128)


In [3]:
from core import run_known_T, parse_T_str


C:\Users\William\desktop\comp5328-Assign2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
print(mean06, std06)

0.5596666666666665 0.03161785712044525
